In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
!ls drive

MyDrive  Othercomputers


In [4]:
%cd '/content/drive/MyDrive/GraphDTA-master-2'

/content/drive/MyDrive/GraphDTA-master-2


# New Section

In [5]:
import torch

In [ ]:
!pip install torch-geometric

In [ ]:
!pip install torch torchvision numpy pandas scikit-learn tqdm matplotlib seaborn

In [ ]:
!pip install rdkit

In [ ]:
!pip install dgl

In [ ]:
import numpy as np
import pandas as pd
import sys, os
from random import shuffle
import torch
import torch.nn as nn
from models.gat import GATNet
from models.gat_gcn import GAT_GATNet
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import *
import matplotlib.pyplot as plt

In [ ]:
#!pip install Tensorflow

In [ ]:
!pip install Scikit-learn
!pip install Matplotlib Seaborn
!pip install subword_nmt periodictable tensorflow_addons


In [11]:
#import tensorflow as tf
import glob
import json
import itertools
import periodictable as pt
import re
from itertools import chain
from operator import itemgetter
from subword_nmt.apply_bpe import BPE
import codecs

In [ ]:
#from tensorflow.python.compiler.tensorrt import trt_convert as trt

In [ ]:
#print("Tensorflow version: ", tf.version.VERSION)

In [12]:
!python '/content/drive/MyDrive/GraphDTA-master-2/create_data.py'

convert data from DeepDTA for  davis1

dataset: davis1
train_fold: 25046
test_fold: 5010
len(set(drugs)),len(set(prots)): 68 379
preparing  davis1_train.pt in pytorch format!
Pre-processed data data/processed/davis1_train.pt not found, doing pre-processing...
Converting SMILES to graph: 1/25046
/content/drive/MyDrive/GraphDTA-master-2/utils.py:68: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  GCNData = DATA.Data(x=torch.Tensor(features),
Traceback (most recent call last):
  File "/content/drive/MyDrive/GraphDTA-master-2/create_data.py", line 292, in <module>
    train_data = TestbedDataset(root='data', dataset=dataset + '_train', xd=train_drugs, xt=train_prots, y=train_Y,
  File "/content/drive/MyDrive/GraphDTA-master-2/utils.py", line 28, in __init__
    self.process(xd, xt,

# **create_data.py**

In [13]:
import pandas as pd
import numpy as np
import os
import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx
from utils import *
import torch as nn
import glob
import json
#import periodictable as pt
import re
from itertools import chain
from operator import itemgetter
from subword_nmt.apply_bpe import BPE
import codecs
#from GraphDTA#######

def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),
                                          ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As',
                                           'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se',
                                           'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
                                           'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    [atom.GetIsAromatic()])


def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)

    c_size = mol.GetNumAtoms()

    features = []
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append(feature / sum(feature))

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])

    return c_size, features, edge_index
    drugs = []
    compound_iso_smiles = []
    for dt_name in ['kiba', 'davis']:
        opts = ['train', 'test']
        for opt in opts:
            df = pd.read_csv('data/' + dt_name + '_' + opt + '.csv')
            compound_iso_smiles += list(df['compound_iso_smiles'])
    compound_iso_smiles = set(compound_iso_smiles)
    smile_graph = {}
    for smile in compound_iso_smiles:
        g = smile_to_graph(smile)
        smile_graph[smile] = g
############## from DTITR ##############################################################
class dataset_builder():
    def __init__(self, data_path, **kwargs):
        super(dataset_builder, self).__init__(**kwargs)
        self.data_path = data_path

    def get_data(self):
        dataset = pd.read_csv(self.data_path['data'], sep=',', memory_map=True)
        prot_dictionary = json.load(open(self.data_path['prot_dic']))
        #smiles_dictionary = json.load(open(self.data_path['smiles_dic']))
        clusters = []
        bpe_codes_prot = ''
        bpe_codes_map_prot = ''

        for i in self.data_path['clusters']:
            if 'test' in i:
                clusters.append(('test', pd.read_csv(i, header=None)))
            else:
                clusters.append(('train', pd.read_csv(i, header=None)))

        if self.data_path['prot_bpe'] != '':
            bpe_codes_prot = codecs.open(self.data_path['prot_bpe'][0])
            bpe_codes_map_prot = pd.read_csv(self.data_path['prot_bpe'][1])

        return (dataset, prot_dictionary, clusters, bpe_codes_prot, bpe_codes_map_prot)

    def data_conversion(self, data, dictionary, max_len):
        keys = list(i for i in dictionary.keys() if len(i) > 1)

        if len(keys) == 0:
            data = pd.DataFrame([list(i) for i in data])

        else:
            char_list = []
            for i in data:
                positions = []
                for j in keys:
                    positions.extend([(k.start(), k.end() - k.start()) for k in re.finditer(j, i)])

                positions = sorted(positions, key=itemgetter(0))

                if len(positions) == 0:
                    char_list.append(list(i))

                else:
                    new_list = []
                    j = 0
                    positions_start = [k[0] for k in positions]
                    positions_len = [k[1] for k in positions]

                    while j < len(i):
                        if j in positions_start:
                            new_list.append(str(i[j] + i[j + positions_len[positions_start.index(j)] - 1]))
                            j = j + positions_len[positions_start.index(j)]
                        else:
                            new_list.append(i[j])
                            j = j + 1
                    char_list.append(new_list)

            data = pd.DataFrame(char_list)

        data.replace(dictionary, inplace=True)

        data = data.fillna(0)
        if len(data.iloc[0, :]) == max_len:
            return data
        else:
            zeros_array = np.zeros(shape=(len(data.iloc[:, 0]), max_len - len(data.iloc[0, :])))
            data = pd.concat((data, pd.DataFrame(zeros_array)), axis=1)
            return data

    def encoding_bpe(self, data, codes, codes_map, max_len):
        bpe = BPE(codes, merges=-1, separator='')
        idx2word = codes_map['index'].values
        words2idx = dict(zip(idx2word, range(0, len(idx2word))))

        vectors = []

        for i in data:
            t1 = bpe.process_line(i).split()  # split
            try:
                i1 = np.asarray([words2idx[j] + 1 for j in t1])  # index
            except:
                i1 = np.array([0])

            l = len(i1)

            if l < max_len:
                k = np.pad(i1, (0, max_len - l), 'constant', constant_values=0)
            else:
                k = i1[:max_len]
            vectors.append(k[None, :])

        return torch.int32(torch.cat(vectors, dim=0))



    def transform_dataset(self, prot_bpe_enc_opt,
                          protein_column,
                          kd_column, bpe_prot_max_len, prot_max_len):

      if prot_bpe_enc_opt == True:

         protein_data = self.encoding_bpe(self.get_data()[0][protein_column], self.get_data()[4],
                                     self.get_data()[5], bpe_prot_max_len)


      else:

         protein_data = torch.tensor(self.data_conversion(self.get_data()[0][protein_column],
                                                     self.get_data()[1], prot_max_len).astype('int32'),
                                dtype=torch.int32)


      kd_values = self.get_data()[0][kd_column].astype('float32')

      #print(protein_data)
      return protein_data, kd_values


all_prots = []
datasets = ['davis1']
for dataset in datasets:
    print('convert data from DeepDTA for ', dataset)
    fpath = 'data/' + dataset + '/'
    train_fold = json.load(open(fpath + "folds/train_fold_setting1.txt"))
    train_fold = [ee for e in train_fold for ee in e]
    valid_fold = json.load(open(fpath +"folds/test_fold_setting1.txt"))
    ligands = json.load(open(fpath + "ligands_can.txt"), object_pairs_hook=OrderedDict)
    proteins = json.load(open(fpath + "proteins.txt"), object_pairs_hook=OrderedDict)
    affinity = pickle.load(open(fpath + "Y", "rb"), encoding='latin1')
    drugs = []
    prots = []
    for d in ligands.keys():
        lg = Chem.MolToSmiles(Chem.MolFromSmiles(ligands[d]), isomericSmiles=True)
        drugs.append(lg)
    for t in proteins.keys():
        prots.append(proteins[t])
    if dataset == 'davis':
        affinity = [-np.log10(y / 1e9) for y in affinity]
    affinity = np.asarray(affinity)
    opts = ['train', 'test']
    for opt in opts:
        rows, cols = np.where(np.isnan(affinity) == False)
        if opt == 'train':
            rows, cols = rows[train_fold], cols[train_fold]
        elif opt == 'test':
            rows, cols = rows[valid_fold], cols[valid_fold]
        with open('data/' + dataset + '_' + opt + '.csv', 'w') as f:
            f.write('compound_iso_smiles,target_sequence,affinity\n')
            for pair_ind in range(len(rows)):
                ls = []
                ls += [drugs[rows[pair_ind]]]
                ls += [prots[cols[pair_ind]]]
                ls += [affinity[rows[pair_ind], cols[pair_ind]]]
                f.write(','.join(map(str, ls)) + '\n')
    print('\ndataset:', dataset)
    print('train_fold:', len(train_fold))
    print('test_fold:', len(valid_fold))
    print('len(set(drugs)),len(set(prots)):', len(set(drugs)), len(set(prots)))
    all_prots += list(set(prots))

compound_iso_smiles = []
for dt_name in ['kiba','davis']:
    opts = ['train','test']
    for opt in opts:
        df = pd.read_csv('data/' + dt_name + '_' + opt + '.csv')
        compound_iso_smiles += list( df['compound_iso_smiles'] )
compound_iso_smiles = set(compound_iso_smiles)
smile_graph = {}
for smile in compound_iso_smiles:
    g = smile_to_graph(smile)
    smile_graph[smile] = g

class dataset_builder:
    def transform_dataset(self, prot_bpe_enc_opt, protein_column, kd_column, bpe_prot_max_len, protein_sequence):
      return protein_sequence

    def get_data(self):
        dataset = pd.read_csv(self.data_path['data'], sep=',', memory_map=True)
        prot_dictionary = json.load(open(self.data_path['prot_dic']))
        # smiles_dictionary = json.load(open(self.data_path['smiles_dic']))
        clusters = []
        bpe_codes_prot = ''
        bpe_codes_map_prot = ''
        return (dataset, prot_dictionary, clusters, bpe_codes_prot, bpe_codes_map_prot)


data_builder = dataset_builder()
#train_Y = list(data_builder.get_data()[0]['affinity'])
prot_bpe_enc_opt = True
protein_column = 'target_sequence'
kd_column = 'affinity'
bpe_prot_max_len = 1000

datasets = ['davis1']
# convert to PyTorch data format
for dataset in datasets:
    processed_data_file_train = 'data/processed/' + dataset + '_train.pt'
    processed_data_file_test = 'data/processed/' + dataset + '_test.pt'
    if ((not os.path.isfile(processed_data_file_train)) or (not os.path.isfile(processed_data_file_test))):
        df = pd.read_csv('data/' + dataset + '_train.csv')
        train_drugs, train_prots,  train_Y = list(df['compound_iso_smiles']),list(df['target_sequence']),list(df['affinity'])
        XT = [data_builder.transform_dataset(prot_bpe_enc_opt, protein_column, kd_column, bpe_prot_max_len, t) for t in train_prots]
        train_drugs, train_prots,  train_Y = np.asarray(train_drugs), np.asarray(XT), np.asarray(train_Y)
        df = pd.read_csv('data/' + dataset + '_test.csv')
        test_drugs, test_prots,  test_Y = list(df['compound_iso_smiles']),list(df['target_sequence']),list(df['affinity'])
        XT = [data_builder.transform_dataset(prot_bpe_enc_opt, protein_column, kd_column, bpe_prot_max_len, t) for t in test_prots]
        test_drugs, test_prots,  test_Y = np.asarray(test_drugs), np.asarray(XT), np.asarray(test_Y)

        # make data PyTorch Geometric ready

        # Convert target values to floats
        train_Y = [float(y) for y in train_Y]

        print('preparing ', dataset + '_train.pt in pytorch format!')
        train_data = TestbedDataset(root='data', dataset=dataset + '_train', xd=train_drugs, xt=train_prots, y=train_Y,
                                    smile_graph=smile_graph)
        print('preparing ', dataset + '_test.pt in pytorch format!')
        test_data = TestbedDataset(root='data', dataset=dataset+'_test', xd=test_drugs, xt=test_prots, y=test_Y,smile_graph=smile_graph)
        print(processed_data_file_train, ' and ', processed_data_file_test, ' have been created')
    else:
        print(processed_data_file_train, ' and ', processed_data_file_test, ' are already created')



convert data from DeepDTA for  davis1

dataset: davis1
train_fold: 25046
test_fold: 5010
len(set(drugs)),len(set(prots)): 68 379
preparing  davis1_train.pt in pytorch format!
Pre-processed data data/processed/davis1_train.pt not found, doing pre-processing...
Converting SMILES to graph: 1/25046


/content/drive/MyDrive/GraphDTA-master-2/utils.py:68: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  GCNData = DATA.Data(x=torch.Tensor(features),


ValueError: ignored

# **training.py**